# Deep Learning Compiler: TVM

## Setup TVM
https://tvm.apache.org/docs/install/from_source.html

In [1]:
import sys
sys.path.append('/Users/likarajo/tvm/python')
sys.path.append('/Users/likarajo/tvm/topi/python')
sys.path.append('/Users/likarajo/tvm/nnvm/python')
sys.path.append('/Users/likarajo/tvm/vta/python')

import tvm

## Setup TensorFlow
https://www.tensorflow.org/install/

## Dowload and extract existing model
http://download.tensorflow.org/models/mobilenet_v1_2018_02_22/mobilenet_v1_1.0_224.tgz.

In [7]:
import tensorflow as tf

cat 'models/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224_info.txt'

Model: mobilenet_v1_1.0_224
Input: input
Output: MobilenetV1/Predictions/Reshape_1


## Import the existing TensorFlow model

In [38]:
import os

with tf.compat.v1.gfile.FastGFile(os.path.join('models/mobilenet_v1_1.0_224/mobilenet_v1_1.0_224_frozen.pb'), 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    graph = tf.import_graph_def(graph_def, name='')

## Obtain the graph definition and shape

In [49]:
try:
    import pytest
except ImportError:
    !pip3 install pytest
    
import tvm.relay.testing.tf as tf_testing
graph_def = tf_testing.ProcessGraphDefParam(graph_def)

shape_dict = {'input': (1, 244, 244, 3)}

## Import graph through frontend
**Relay** comes as a submodule of TVM. 
* It is a high level intermediate representation for the TVM framework. 
* It replaces old computation graph based IRs such as NNVM with a more expressive IR that can be effectively optimized for many targets.

In [50]:
"""
sym : tvm symbol graph constructed from imported model
params : graph parameters imported from model
"""
sym, params = tvm.relay.frontend.from_tensorflow(graph_def, shape=shape_dict)

## Compile the model for LLVM target

In [52]:
"""
target: the compiler to build the output for
"""
graph, lib, params = tvm.relay.build(sym, target='llvm', params=params)

AttributeError: Traceback (most recent call last):
  [bt] (8) 9   libtvm.dylib                        0x0000000109c58007 tvm::relay::ExprMutator::VisitExpr_(tvm::relay::FunctionNode const*) + 903
  [bt] (7) 8   libtvm.dylib                        0x0000000109c56af9 tvm::relay::MixedModeMutator::VisitExpr(tvm::RelayExpr const&) + 761
  [bt] (6) 7   libtvm.dylib                        0x0000000109c563fb tvm::relay::MixedModeMutator::VisitLeaf(tvm::RelayExpr const&) + 331
  [bt] (5) 6   libtvm.dylib                        0x0000000109c5dbb7 tvm::relay::PostOrderRewriter::DispatchVisitExpr(tvm::RelayExpr const&) + 55
  [bt] (4) 5   libtvm.dylib                        0x0000000109a0045d tvm::relay::ExprRewriter::Rewrite(tvm::RelayExpr const&, tvm::RelayExpr const&) + 189
  [bt] (3) 4   libtvm.dylib                        0x0000000109a00d22 tvm::NodeFunctor<tvm::RelayExpr (tvm::runtime::ObjectRef const&, tvm::relay::ExprRewriter*, tvm::RelayExpr const&)>::operator()(tvm::runtime::ObjectRef c